In [1]:
import gentrl
import torch
import pandas as pd
import time
from torch.utils.data import DataLoader
torch.cuda.set_device(3)

In [2]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda();

In [6]:
md = gentrl.MolecularDataset(sources=[
    {'path':'train_subset_100_000.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'plogP' : 'plogP',
    }], 
    props=['plogP'])

In [13]:
batch_size = 512
num_workers = 1
pin_memory = False
lr = 1e-4
num_epochs = 1

In [14]:
train_loader = DataLoader(md, batch_size=batch_size,
                          shuffle=True, num_workers=num_workers,
                          pin_memory=pin_memory, drop_last=True)

In [15]:
start_time = time.time()
model.train_as_vaelp(train_loader, num_epochs=num_epochs, lr=lr)
end_time = time.time()
duration = end_time - start_time
print(f"Time Taken is {duration/60} min")

Epoch 0 :
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.9831;rec: -0.915;kl: -19.6;log_p_y_by_z: -0.8772;log_p_z_by_y: -41.76;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.9351;rec: -0.8911;kl: -18.52;log_p_y_by_z: -0.6247;log_p_z_by_y: -39.84;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.9084;rec: -0.8679;kl: -17.11;log_p_y_by_z: -0.577;log_p_z_by_y: -39.75;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.8856;rec: -0.848;kl: -16.09;log_p_y_by_z: -0.5366;log_p_z_by_y: -39.57;
Time Taken is 1.7872628092765808 min


In [ ]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol


def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [ ]:
! mkdir -p saved_gentrl

In [ ]:
model.save('./saved_gentrl/')